In [1]:
'''
对T+CLOUD上的收发存汇总表进行二次开发，与公司以前的格式一致
'''
import os
import numpy as np
import easygui
import pandas as pd
import openpyxl

In [3]:
filename = '收发存汇总表.xlsx'
wb = openpyxl.Workbook()
wb.save(filename)

lst = [
'存货分类 (1级)',
 '存货分类 (2级)',
 '存货分类 (3级)',
 '存货分类 (4级)',
 '存货分类 (5级)',
 '存货编码',
 '存货',
 '存货代码',
  '数量(主单位)',
 '平均单价',
 '金额',
 '数量(辅单位)',
 '计量单位组合',
 '数量(主单位).1',
 '平均单价.1',
 '金额.1',
 '数量(辅单位).1',
 '计量单位组合.1',
 '数量(主单位).2',
 '平均单价.2',
 '金额.2',
 '数量(辅单位).2',
 '计量单位组合.2',
 '数量(主单位).3',
 '平均单价.3',
 '金额.3',
 '辅数量',
 '计量单位组合.3']

lst2 = [
'class01',
 'class02',
 'class03',
 'class04',
 'class05',
 'code',
 'stock',
 'content',
  'begin_ben',
 '平均单价',
 '金额',
 'begin_jian',
 '计量单位组合',
 'ruku_ben',
 '平均单价.1',
 '金额.1',
 'ruku_jian',
 '计量单位组合.1',
 'chuku_ben',
 '平均单价.2',
 '金额.2',
 'chuku_jian',
 '计量单位组合.2',
 'end_ben',
 '平均单价.3',
 '金额.3',
 'end_jian',
 '计量单位组合.3']

lst3 =  [
 'class02',
 'class05',
 'code',
 'stock',
 'content',
 'begin_jian',
 'ruku_jian',
 'chuku_jian',
 'end_jian',
]

lst01_yesterday =  [
 'class02',
 'class05',
 'code',
 'stock',
 'begin_jian',
 'ruku_jian',
 'chuku_jian',
]
lst02_yesterday =  [
 'class02',
 'class05',
 'code',
 'stock',
 'yesterday_jian',
 'yesterday_ruku',
 'yesterday_chuku',

]
lst_merge = ['class02','class05','code','stock',
             'content',
             'begin_jian','yesterday_jian',
             'yesterday_ruku','ruku_jian',
              'yesterday_chuku','chuku_jian',
               'end_jian']

lst_result0 = ['class05','code','stock',
             'content',
             'begin_jian','yesterday_jian',
             'yesterday_ruku','ruku_jian',
              'yesterday_chuku','chuku_jian',
               'end_jian']

lst_result = ['类别','货号','品名',
             '含量',
             '月初','上日',
             '本日入库','入库累计',
              '本日出库','出库累计',
               '结余']

def chuli(fname):
    df =  pd.read_excel(fname,skiprows = 7)
    df = df.iloc[:,1:]
    df.columns = lst
    df.columns = lst2
    df['content'] = df['end_ben']/df['end_jian']
    df = df[df['class01'] != '制表人:']
    df = df[df['class01'] != '合计：']
    df = df[df['class01'].notnull()]
    df = df[lst3]
    return df
    
   

In [4]:
fname = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表01.xlsx"
fname2 = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表02.xlsx"
df1 = chuli(fname)
df2 = chuli(fname2)
df1 = df1[lst01_yesterday]
df1.columns = lst02_yesterday

In [6]:
merge = pd.merge(df2,df1,how = 'left',on = ['class02','class05','code','stock']) 
gp = merge.groupby('class02')
def xiaojiClass05(gp2,key,d):
    d = d[lst_result0]
    d.loc[key] = d.sum()
    d.iloc[-1,:4] = [key + '小计','','','']
    return d

In [7]:
merge = pd.merge(df2,df1,how = 'left',on = ['class02','class05','code','stock']) 
gp = merge.groupby('class02')
def xiaojiClass05(gp2,key,d):
    d = d[lst_result0]
    d.loc[key] = d.sum()
    d.iloc[-1,:4] = [key + '小计','','','']
    return d
for name,gp002 in gp: 
    table = pd.pivot_table(gp002,index = ['class05','code'],aggfunc = sum)
    table = table.reset_index()
    gp2 = table.groupby('class05')
    
    heji = pd.DataFrame(gp002).sum()
    data = []
    for key,gp005 in gp2:
        df05 = xiaojiClass05(gp2,key,gp005)
        data.append(df05)
    df005 = pd.concat(data)
    df005 = df005[lst_result0]
    df005.loc[name] = heji
    df005.iloc[-1,:4] = [name + '合计','','','']
    with pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        df005.to_excel(writer, sheet_name=name, index=False)

        

C:\Users\redda\AppData\Local\Temp\ipykernel_27152\1980026135.py:2: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  table = pd.pivot_table(gp002,index = ['class05','code'],aggfunc = sum)
C:\Users\redda\AppData\Local\Temp\ipykernel_27152\3663803954.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.loc[key] = d.sum()
C:\Users\redda\AppData\Local\Temp\ipykernel_27152\3663803954.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  d.iloc[-1,:4] = 